<a href="https://colab.research.google.com/github/hruthiksiva/iith-competition/blob/main/iith_deepn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load 

In [ ]:
!gdown https://drive.google.com/u/0/uc?id=1fgv0AUuIhyqJ_8DW54LYJlW58x8_ekGY
!unzip -qq "deepn.zip"

Downloading...
From: https://drive.google.com/u/0/uc?id=1fgv0AUuIhyqJ_8DW54LYJlW58x8_ekGY
To: /content/deepn.zip
100% 453k/453k [00:00<00:00, 50.9MB/s]


# mrLittle

## imports

In [ ]:
import pandas as pd
import numpy as np
import re

## Load Data

In [ ]:
df = pd.read_csv('Train_Data.csv')
df2 = pd.read_csv('Test_Data.csv')

## Pre-Processing

In [ ]:
def finder(str1):
    temp1 = re.compile('http://t.co/..........')
    temp2 = re.compile('#[a-zA-Z0-9_]+')
    temp3 = re.compile('@[a-zA-Z0-9_]+')
    links = []
    tags = []
    mentions = []
    str2 = str1
    for match in temp1.finditer(str1):
        links.append(match.group())
        str2 = str2.replace(match.group(),'')
    for match in temp2.finditer(str1):
        tags.append(match.group())
        str2 = str2.replace(match.group(),'')
    for match in temp3.finditer(str1):
        mentions.append(match.group())
        str2 = str2.replace(match.group(),'')
    links_ = len(links)
    tags_ = len(tags)
    mentions_ = len(mentions)
    links = None if not links else links
    tags = None if not tags else tags
    mentions = None if not mentions else mentions
    return str2, links, tags, mentions, links_, tags_, mentions_

In [ ]:
for i in range(2,-1,-1):
  df.loc[(df[f'Unnamed: 1{i}'].notnull()),'text'] = df.loc[(df[f'Unnamed: 1{i}'].notnull())].iloc[:,1:i+3].agg(''.join, axis=1)
  df.iloc[(df[f'Unnamed: 1{i}'].notnull()),2:] = df.loc[(df[f'Unnamed: 1{i}'].notnull())].iloc[:,2:].shift(periods=-i-1, axis=1)
df.drop(columns=['Unnamed: 10','Unnamed: 11','Unnamed: 12'], inplace = True)

for i in range(14,8,-1):
  df2.loc[(df2[f'Unnamed: {i}'].notnull()),'text'] = df2.loc[(df2[f'Unnamed: {i}'].notnull())].iloc[:,1:i-3].agg(''.join, axis=1)
  df2.iloc[(df2[f'Unnamed: {i}'].notnull()),2:] = df2.loc[(df2[f'Unnamed: {i}'].notnull())].iloc[:,2:].shift(periods=-i+8, axis=1)
df2.drop(columns=['Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14'], inplace = True)

In [ ]:
df.drop(df.loc[df['header'] == '1'].index, inplace = True)
df.loc[(df['rumour_identification'] == 1.0),'rumour_identification'] = '1' 

In [ ]:
df.drop(columns=['type_of_tweet','tweet_language'], inplace = True)
df2.drop(columns=['type_of_tweet','tweet_language'], inplace = True)

In [ ]:
df[['Identity','num_followers','num_friends','num_retweets','rumour_identification']] = df[['Identity','num_followers','num_friends','num_retweets','rumour_identification']].astype('int64')
df2[['Identity','num_followers','num_friends','num_retweets']] = df2[['Identity','num_followers','num_friends','num_retweets']].astype('int64')

In [ ]:
df[['screen', 'username']] = df['screen_and_username'].str.rsplit(pat='/', n=1, expand=True)
df.drop(columns=['screen_and_username'], inplace=True)
df2[['screen', 'username']] = df2['screen_and_username'].str.rsplit(pat='/', n=1, expand=True)
df2.drop(columns=['screen_and_username'], inplace=True)

In [ ]:
df[['text','links','tags','mentions','nolinks','notags','nomentions']]=df.text.apply(finder).tolist()
df2[['text','links','tags','mentions','nolinks','notags','nomentions']]=df2.text.apply(finder).tolist()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
def yesno(obj):
    if obj not in temp_dict:
        return 0, 0
    return temp_dict[obj]['yes'], temp_dict[obj]['no']
temp_dict = {}
for name,rumour in zip(df['username'],df['rumour_identification']):
    if name not in temp_dict:
        temp_dict[name] = {'yes' : 0, 'no' : 0}
    if rumour :
        temp_dict[name]['yes'] = temp_dict[name]['yes'] + 1
    else:
        temp_dict[name]['no'] = temp_dict[name]['no'] + 1
df[['yes', 'no']] = df['username'].apply(yesno).to_list()
df['yes-no'] = df['yes']-df['no']
df['yes-no-sq'] = pow(df['yes-no'],3)
df2[['yes', 'no']] = df2['username'].apply(yesno).to_list()
df2['yes-no'] = df2['yes']-df2['no']
df2['yes-no-sq'] = pow(df2['yes-no'],3)

In [ ]:
df.to_csv('Train.csv', index=False)
df2.to_csv('Test.csv', index=False)

## display

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4060 entries, 0 to 4060
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Identity               4060 non-null   int64 
 1   text                   4060 non-null   object
 2   header                 4060 non-null   object
 3   num_followers          4060 non-null   int64 
 4   num_friends            4060 non-null   int64 
 5   num_retweets           4060 non-null   int64 
 6   rumour_identification  4060 non-null   int64 
 7   screen                 4060 non-null   object
 8   username               4060 non-null   object
 9   links                  1508 non-null   object
 10  tags                   1068 non-null   object
 11  mentions               3480 non-null   object
dtypes: int64(5), object(7)
memory usage: 412.3+ KB


In [ ]:
df[df['rumour_identification']==1].describe(include='all')

,Identity,text,header,num_followers,num_friends,num_retweets,rumour_identification,screen,username
count,2698.000000,2698,2698,2.698000e+03,2698.000000,2698.000000,2698.0,2698,2698
unique,NaN,2694,5,NaN,NaN,NaN,NaN,1559,1560
top,NaN,"Thinking of Layton's words: ""Love is better th...",charliehebdo,NaN,NaN,NaN,NaN,Antonio French,AntonioFrench
freq,NaN,2,1135,NaN,NaN,NaN,NaN,72,72
mean,2855.032246,NaN,NaN,9.348404e+05,4285.640845,453.012602,1.0,NaN,NaN
std,1631.140000,NaN,NaN,2.727699e+06,21120.736327,2619.116841,0.0,NaN,NaN
min,458.000000,NaN,NaN,1.900000e+01,0.000000,26.000000,1.0,NaN,NaN
25%,1430.250000,NaN,NaN,8.334250e+03,367.000000,130.000000,1.0,NaN,NaN
50%,2660.500000,NaN,NaN,5.140400e+04,886.000000,186.000000,1.0,NaN,NaN
75%,4307.250000,NaN,NaN,3.629245e+05,1766.000000,346.000000,1.0,NaN,NaN


In [ ]:
df1=

In [ ]:
df.head()

,Identity,text,header,num_followers,num_friends,num_retweets,rumour_identification,screen,username,links,tags,mentions,nolinks,notags,nomentions
0,2617,Statement from St. Louis County Police Officer...,ferguson,1571,1463,465,1,Alex Heuer,alexheuer,['http://t.co/plnDpI5uRE'],['#Ferguson'],['@GovJayNixon'],1,1,1
1,5075,Australian radio reporting that gunman not on...,sydneysiege,17860,6864,175,0,Jack Moore,JFXM,NaN,['#SydneySiege'],NaN,0,1,0
2,1293,: Hundreds of Muslims gather outside the Grand...,charliehebdo,36919,242,815,1,AFP Photo Department,AFPphoto,['http://t.co/LeC7StybFn'],['#CharlieHebdo'],NaN,1,1,0
3,5251,Authorities have confirmed that is now over a...,sydneysiege,36137,926,117,1,Yahoo UK News,YahooNewsUK,"['http://t.co/IrteddYrbH', 'http://t.co/w8i1nj...",['#sydneysiege'],NaN,2,1,0
4,4850,One man has been arrested and people are bein...,sydneysiege,169840,1421,264,0,ABC News 24,ABCNews24,NaN,"['#BREAKING', '#sydneysiege', '#martinplace']",NaN,0,3,0


# arena

In [ ]:
temp_dict

{'alexheuer': {'no': 0, 'yes': 1},
 'JFXM': {'no': 5, 'yes': 4},
 'AFPphoto': {'no': 9, 'yes': 7},
 'YahooNewsUK': {'no': 0, 'yes': 2},
 'ABCNews24': {'no': 4, 'yes': 1},
 'BreakingNews': {'no': 19, 'yes': 10},
 'ProPublica': {'no': 0, 'yes': 2},
 'CityNews': {'no': 0, 'yes': 1},
 'nlynnbogart': {'no': 0, 'yes': 1},
 'RT_com': {'no': 41, 'yes': 25},
 'BBCBreaking': {'no': 38, 'yes': 18},
 'TarekFatah': {'no': 0, 'yes': 9},
 'CP24': {'no': 13, 'yes': 7},
 'SkyNews': {'no': 28, 'yes': 32},
 'TPM': {'no': 3, 'yes': 2},
 'abcnews': {'no': 11, 'yes': 8},
 'flightradar24': {'no': 5, 'yes': 7},
 'popsychoIogy': {'no': 0, 'yes': 1},
 'NBCNews': {'no': 18, 'yes': 9},
 'fallongreen15': {'no': 0, 'yes': 1},
 'erichbergen': {'no': 1, 'yes': 0},
 'TelegraphNews': {'no': 5, 'yes': 3},
 'AnthonyCumia': {'no': 0, 'yes': 3},
 'SalafiSisters': {'no': 0, 'yes': 1},
 'peddoc63': {'no': 0, 'yes': 4},
 'MartinDaubney': {'no': 0, 'yes': 2},
 'ClassicPict': {'no': 0, 'yes': 1},
 'noel_fisher': {'no': 0, 'yes'

In [ ]:
df = pd.read_csv('Train.csv')
df2 = pd.read_csv('Test.csv')

In [ ]:
import re
sub_str = 'http://t.co/..........'
temp = re.compile(sub_str)  
res = temp.search('Shooting at #CharlieHebdo "an attack on freedom", says Dominque Moisi, Parisian political analyst: http://t.co/B3AvGBwp82')
res 

In [ ]:
df2.head()

,Identity,text,header,num_followers,num_friends,num_retweets,screen,username,links,tags,mentions,nolinks,notags,nomentions
0,5070,Black flag held by hostages bears Islamic mes...,sydneysiege,4637013,382,232,Fox News,FoxNews,"['http://t.co/rEpn2Mt4UC', 'http://t.co/U9sqDm...",['#Sydneysiege'],NaN,2,1,0
1,1103,The latest on the manhunt for Charlie Hebdo at...,charliehebdo,5586173,976,158,Wall Street Journal,WSJ,"['http://t.co/A3ChSCOHck', 'http://t.co/ArkKqS...",NaN,NaN,2,0,0
2,812,"The ""appropriate"" response to is not to urge ...",charliehebdo,14110,650,333,Archbishop Cranmer,His_Grace,['http://t.co/lOPLInljML'],"['#CharlieHebdo', '#JeSuisCharlie']",NaN,1,2,0
3,1632,"Dual hostage crisis ends: suspects killed, ‘s...",charliehebdo,1261668,381,285,FRANCE 24,FRANCE24,"['http://t.co/LdSF5QeCiv', 'http://t.co/Gm5z8o...",['#CharlieHebdo'],NaN,2,1,0
4,1128,Cartoonists Georges Wolinski and Cabu are amon...,charliehebdo,702812,360,225,AJE News,AJENews,"['http://t.co/EJ0gGFsRZ1', 'http://t.co/42xHoE...",['#CharlieHebdo'],NaN,2,1,0


In [ ]:
df.head(40000)

,Identity,text,header,num_followers,num_friends,num_retweets,rumour_identification,screen,username
0,2617,Statement from St. Louis County Police Officer...,ferguson,1571,1463,465,1,Alex Heuer,alexheuer
1,5075,Australian radio reporting that #SydneySiege g...,sydneysiege,17860,6864,175,0,Jack Moore,JFXM
2,1293,#CharlieHebdo: Hundreds of Muslims gather outs...,charliehebdo,36919,242,815,1,AFP Photo Department,AFPphoto
3,5251,Authorities have confirmed that #sydneysiege i...,sydneysiege,36137,926,117,1,Yahoo UK News,YahooNewsUK
4,4850,#BREAKING One man has been arrested and people...,sydneysiege,169840,1421,264,0,ABC News 24,ABCNews24
...,...,...,...,...,...,...,...,...,...
4055,3772,#Canada journalist captures outbreak of small-...,ottawashooting,20416,1059,122,0,Charles Lister,Charles_Lister
4056,5191,Absolutely shocking what Zayn Malik is doing i...,sydneysiege,2196,1238,135,1,S G,SGTYMB
4057,5226,BREAKING: Terrorist involved in Sydney siege i...,sydneysiege,129863,30456,194,1,The Int. Spectator,INTLSpectator
4058,5390,My thoughts are with you Sydney ❤️,sydneysiege,655808,1039,888,1,Lilly Singh,IISuperwomanII


## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense


# Data Preparation

In [ ]:
df = pd.read_csv('Train_Data.csv')
test=pd.read_csv('Test_Data.csv')

In [ ]:
dataset=df.values

# Data Preprocessing

In [ ]:
df['rumour_identification'].unique()

array(['1', '0', 'en', '426', '370', '139', '8623', '717', '142', '146',
       '154', '123', '312', '134', '226', '314', '127'], dtype=object)

In [ ]:
trainX = dataset[:,0:9]

In [ ]:
df[

In [ ]:
df[df['Unnamed: 10'].notnull()]

,Identity,text,header,type_of_tweet,screen_and_username,num_followers,num_friends,num_retweets,tweet_language,rumour_identification,Unnamed: 10,Unnamed: 11,Unnamed: 12
46,1024,Yes this is about victims. It's also about how...,abolish blasphemy codes. We mustn't run from ...,charliehebdo,1,Maajid Nawaz/MaajidNawaz,33230,363,120,en,1,NaN,NaN
68,5439,#Coquitlam crews are on Sydney Ave after a mal...,smoking a cigarette,sydneysiege,1,ScanBC/ScanBC,72226,666,211,en,1,NaN,NaN
81,1225,One of the most compelling &amp,relevant things I've seen on #CharlieHebdo so...,charliehebdo,1,Ryan Rampling/RyanRampling,2284,2071,287,en,1,NaN,NaN
94,3875,Police in pursuit of multiple shooters in Ottawa,PM safe,downtown buildings in lockdown http://t.co/3F...,ottawashooting,1,The Globe and Mail/globeandmail,534910,409,426,en,0,NaN
117,2366,Brutal treatment of black ppl isn’t indeed the...,look at US’s green light to #Israel’s crimes. ...,ferguson,1,Khamenei.ir/khamenei_ir,69710,3,223,en,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3992,1021,I'm not #CharlieHebdo &amp,neither are the French or anyone else unless ...,charliehebdo,1,Shuja Rabbani/ShujaRabbani,138235,1980,1157,en,1,NaN,NaN
3999,4014,Watch live: CBC NN coverage of shooting of sol...,shooter still on loose around Parliament Hill...,ottawashooting,1,CBC News Alerts/CBCAlerts,365767,264,394,en,0,NaN,NaN
4017,3556,Our thoughts &amp,prayers are with the families and friends of ...,crew.,germanwings-crash,1,IATA/IATA,108290,763,127,en,1,NaN
4033,2904,If u r white &amp,u r not outraged about #Ferguson then the pro...,ferguson,1,Muriel B/QuiltingMuriel,49956,5300,134,en,1,NaN,NaN


In [ ]:
trainY=dataset[:,9]

In [ ]:
trainY

array(['1', '0', '1', ..., '1', '1', '1'], dtype=object)

In [ ]:
X_val, X_test, Y_val, Y_test = train_test_split(trainX, trainY, test_size=0.5)

In [ ]:
X_val

array([[3174,
        'Despite the latest blunder of the #Ferguson police department, I am hopeful for another night of #PeaceInFerguson.',
        'ferguson', ..., '1041', '390', 'en'],
       [3083,
        '#Ferguson police ID officer who shot and killed #MikeBrown as Darren Wilson.',
        'ferguson', ..., '150', '128', 'en'],
       [3146,
        'There will be no #BlackOut or Censoring The Global Community from witnessing this #HUMAN travesty #Unity #Ferguson http://t.co/LcOVHIHivD',
        'ferguson', ..., '54469', '375', 'en'],
       ...,
       [767,
        "Anjem Choudary claims: 'Muslims don't believe in freedom of speech' after #CharlieHebdo attack http://t.co/V8954pwRoy http://t.co/MZQSRSFZrq",
        'charliehebdo', ..., '1763', '271', 'en'],
       [1933,
        'Charlie Hebdo, the magazine that dared to poke fun at religion http://t.co/L5wXIOOr2p http://t.co/SpDK2rMW4K',
        'charliehebdo', ..., '3311', '105', 'en'],
       [4709, 'SHOTS FIRED AT SYDNEY CAFE

In [ ]:
print(trainX.shape, X_val.shape, X_test.shape, trainY.shape, Y_val.shape, Y_test.shape)

(4061, 9) (2030, 9) (2031, 9) (4061,) (2030,) (2031,)


# Model Designing

In [ ]:
model = Sequential([    Dense(32, activation='relu', input_shape=(10,)),    Dense(32, activation='relu'),    Dense(1, activation='sigmoid'),])

In [ ]:
model.compile(optimizer='sgd',              loss='binary_crossentropy',              metrics=['accuracy'])

In [ ]:
hist = model.fit(trainX, trainY,batch_size=32, epochs=100,validation_data=(X_val, Y_val))

ValueError: ignored

## Prediction

In [ ]:
model.fit()

## Submission

In [ ]:
model.submit()